#### Import libraries

In [12]:
import sys
sys.path.append('BEHRT/Early_integration')
sys.path.append('BEHRT/')
#sys.path.append(cwd+'/Early_integration')
#sys.path.append(cwd)

import random
from Utils.common import *
from Utils.pytorch import load_model
from Utils.utils import age_vocab, input_vocab, mod_vocab, delay_vocab
from Utils.optimiser import adam
from Models.MLM_v2 import MLMLoader
import pytorch_pretrained_bert as Bert
from Models.MLM_v2 import MLMLoader
from torch.utils.data import DataLoader
from Models.MLM import BertForMaskedLM
import pandas as pd
import sklearn.metrics as skm
import numpy as np
import torch
import torch.nn as nn
import os
import time
import warnings
warnings.filterwarnings(action='ignore')
from torch.utils.data.dataset import Dataset
from Utils.dataLoader_utils import seq_padding,position_idx,index_seg,random_mask
from Utils.add_endpoints import add_endp
from Utils.handle_file import handle_file
from transformers.models.bert.modeling_bert import BertPreTrainedModel, BertEncoder, BertPooler, BertOnlyMLMHead
from transformers.modeling_utils import PreTrainedModel
from Utils.handle_file import handle_file
hf = handle_file()
from Utils.preprocessing import cal_age, cal_therapies, spark_init, cal_subtherapies, int_to_list, flatten_list, int_to_str, cal_therapies_fr, divide_frame
from transformers.modeling_outputs import BaseModelOutputWithPoolingAndCrossAttentions, MaskedLMOutput

import torch.nn as nn
import pytorch_pretrained_bert as Bert
import numpy as np
import torch
from typing import Optional, Tuple, Union



#### Perform Masked Language Model 

1. Utils 

In [ ]:
from torch.utils.data.dataset import Dataset
import numpy as np
import torch


def get_delim_for_chunks(sequence, n):
    """Compute chunks of sequence that will be use to predict delay
    Take as input the sequence and the number of visits we want to consider for the chunk
    return the chunked sequence for the task and the real delay following that sequence """
    np.random.seed(seed=2)
    
    num_of_visits=0
    sep_index = [0]  ## start with the first CLS so that when we will choose random start, the first vist can be part of the choices

    for i in range(len(sequence)): 
        if sequence[i] == 'SEP': 
            num_of_visits+=1
            sep_index.append(i+1)
            
    if n >= num_of_visits:
        n = num_of_visits -1 

    start = np.random.choice(sep_index[:len(sep_index)-1-(n)], 1)[0]  ### Start for the chunk must be from 0 to n step before the end of the sep_sequence (in order to have n visits at least from the start) and - 1 is for the last SEP which is the end of the sequence that can't be taken into account to predict delay (bc it's the last visit)
    stop = sep_index[sep_index.index(start) + n]
    
    return start, stop

def get_visits_for_delay_prediction(sequence, n, token2idx):
    start, stop = get_delim_for_chunks(sequence, n)
    
    chunk = sequence[start+1:stop]
    delay = sequence[stop]
    
    output_token = []
    for i, token in enumerate(chunk):
        output_token.append(token2idx.get(token, token2idx['UNK']))
        
    return chunk, output_token, delay
    
def get_other_subsequences(sequence, other_sequence, n):
   
    start, stop = get_delim_for_chunks(sequence, n)
    chunk = other_sequence[start+1:stop]
    return chunk
      
    
class PretrainLoader(Dataset):
    def __init__(self,  token2idx, mod2idx, max_len, dataframe, mlb=None, code='inputs', age='age', mod = 'modalities', delay = 'delays_in_months',  age2idx = None, del2idx=None, age_in_inputs = False, delays_in_inputs = False, right_percentage = 1, split = False, baseline=False, seed = 42, pretrain='NDP', n = 5):
        self.baseline= baseline
        self.vocab = token2idx
        self.mod2idx = mod2idx
        self.max_len = max_len
        self.split = split
        self.r_percentage = right_percentage
        self.seed = seed
        self.code = dataframe[code]
        self.modalities = dataframe[mod]
        self.age_in_inputs = age_in_inputs
        self.delays_in_inputs = delays_in_inputs
        self.n = n
        self.pretrain = pretrain
        self.del2idx = del2idx
        self.age2idx = age2idx
        self.mlb = mlb
        if age2idx is not None: 
            self.age = dataframe[age]
            
        if del2idx is not None:
            self.delay = dataframe[delay]
        

    def __getitem__(self, index):
        """
        return: age, code, position, segmentation, mask, label
        """
        
        if self.pretrain == 'MLM':

            # extract data
            code = self.code[index][(-self.max_len+1):]
            modalities = self.modalities[index][(-self.max_len+1):]
        
            mask = np.ones(self.max_len)
            mask[len(code):] = 0
        
            # pad age sequence and code sequence
            modalities = seq_padding(modalities, self.max_len, self.split, self.r_percentage, token2idx=self.mod2idx)

            # put mask in input sequence
            # token : unchanged sequence, code :  index of sequence with masked token = 3, label: -1 for unmasked token and label index for masked token 
            tokens, code, label = random_mask(code, self.vocab, self.seed)
    
           # get position code and segment code
            tokens = seq_padding(tokens, self.max_len, self.split, self.r_percentage)
            position = position_idx(tokens)
            segment = index_seg(tokens)

           # pad code and label
            code = seq_padding(code, self.max_len, self.split, self.r_percentage, symbol=self.vocab['PAD'])
            label = seq_padding(label, self.max_len, self.split, self.r_percentage, symbol=-1)
            
            if self.del2idx is not None:
                delay = self.delay[index][(-self.max_len+1):]
                delay = seq_padding(delay, self.max_len, self.split, self.r_percentage, token2idx=self.del2idx)
            
            if self.age2idx is not None: 
                age = self.age[index][(-self.max_len+1):]
                age = seq_padding(age, self.max_len, self.split, self.r_percentage, token2idx=self.age2idx)
              
            
            if self.delays_in_inputs: 
                if self.baseline:
                    for col in [code, modalities, segment, mask, label]:
                        random.Random(42).shuffle(col)

                return torch.LongTensor(code), torch.LongTensor(modalities), torch.LongTensor(segment), torch.LongTensor(position), torch.LongTensor(mask), torch.LongTensor(label)
            
            elif self.age_in_inputs:  
                if self.baseline:
                    for col in [code, modalities, delay, segment, mask, label]:
                        random.Random(42).shuffle(col)
                return torch.LongTensor(code), torch.LongTensor(modalities), torch.LongTensor(delay), torch.LongTensor(segment), torch.LongTensor(position), torch.LongTensor(mask), torch.LongTensor(label)
          
            else:
                return torch.LongTensor(code), torch.LongTensor(modalities), torch.LongTensor(age), torch.LongTensor(delay), torch.LongTensor(segment), torch.LongTensor(position), torch.LongTensor(mask), torch.LongTensor(label)

        elif self.pretrain == 'NDP':
            
            code = self.code[index]
        
            modalities = self.modalities[index]
            modalities = get_other_subsequences(code, modalities, n = self.n)
            modalities = seq_padding(modalities, self.max_len, token2idx=self.mod2idx)
            
            NPI = self.NPI[index]
            NPI = get_other_subsequences(code, NPI, n = self.n)
            NPI = seq_padding(NPI, self.max_len, token2idx=self.NPI2idx)
            
            if self.del2idx is not None: 
                delay = self.delay[index]
                delay = get_other_subsequences(code, delay, n=self.n)
                delay = seq_padding(delay, self.max_len, token2idx=self.del2idx)
                
            if self.age2idx is not None:
                age = self.age[index]
                age = get_other_subsequences(code, age, n= self.n)
                age = seq_padding(age, self.max_len, token2idx=self.age2idx)
                
            tokens, code, label = get_visits_for_delay_prediction(code, n=self.n, token2idx=self.vocab)
            
            mask = np.ones(self.max_len)
            mask[len(code):] = 0
            
            # get position code and segment code
            tokens = seq_padding(tokens, self.max_len, self.split, self.r_percentage)
            position = position_idx(tokens)
            segment = index_seg(tokens)
            
            code = seq_padding(code, self.max_len, self.split, self.r_percentage, symbol=self.vocab['PAD'])
            
            label = torch.LongTensor(self.mlb.transform([[label]]))
            
            if self.delays_in_inputs: 
                return torch.LongTensor(code), torch.LongTensor(modalities), torch.LongTensor(segment), torch.LongTensor(position), torch.LongTensor(NPI), torch.LongTensor(mask), label
            
            elif self.age_in_inputs:  
                return torch.LongTensor(code), torch.LongTensor(modalities), torch.LongTensor(delay), torch.LongTensor(segment), torch.LongTensor(position), torch.LongTensor(NPI), torch.LongTensor(mask), label
            
            else:
                return torch.LongTensor(code), torch.LongTensor(modalities), torch.LongTensor(age), torch.LongTensor(delay), torch.LongTensor(segment), torch.LongTensor(position), torch.LongTensor(NPI), torch.LongTensor(mask), label
 

    def __len__(self):
        return len(self.code)
    
## Datasets
def import_datasets(config,  min_visit , inputs= "inputs_normal_range_and_delta_age_and_deltas_preprocessed_tolist"):
    
    """Import train data """
        
    data =  hf._load_pkl(config['data'])[0]

    
    """Remove patients that have less visits than min visit"""
    # Remove patients with visits less than min visit

    previous_shape = data.shape[0]

    data['length'] = data[inputs].apply(lambda x: len([i for i in range(len(x)) if x[i] == 'SEP']))
    data = data[data['length'] >= min_visit]
    data = data.reset_index(drop=True)

    print("We lost {} patient(s)".format(previous_shape - data.shape[0]))
    
    """split data into train and validation data"""
    train, valid = np.split(data.sample(frac=1, random_state=42), [int(.9*len(data))])
    train.index = range(len(train))
    valid.index = range(len(valid))
    
    return train, valid

## Scores
""" Calculate precision for the NDP """
def compute_precision(logits, labels, training=True):
    """
    Compute score for the pretraining task which represents precision:
    Take as input logits in probabilities and labels in a 2D array
    Return: precision score
    
    """
    
    if training:
        labels, logits = labels.detach().cpu().numpy(), logits.detach().cpu().numpy()
        
    logs = nn.LogSoftmax(dim = 1)
 
    truepred = logs(torch.tensor(logits))
    outs = np.zeros(labels.shape)
    ind = [np.argmax(pred_x) for pred_x in truepred.numpy()]

    for i in range(len(ind)):
        outs[i][ind[i]] = 1
    
    precision = precision_score(labels, outs, average='macro')

    return precision


""" Calculate precision for the MLM """

def cal_acc(label, pred):
    logs = nn.LogSoftmax(dim = 1)
    label=label.cpu().numpy()
    ind = np.where(label!=-1)[0]
    truepred = pred.detach().cpu().numpy()
    truepred = truepred[ind]
    truelabel = label[ind]
    truepred = logs(torch.tensor(truepred))
    outs = [np.argmax(pred_x) for pred_x in truepred.numpy()]
    precision = precision_score(truelabel, outs, average='macro')
    return precision

## 3. MISC

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(timedelta(seconds=elapsed_rounded))


def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    


In [ ]:

class BertEmbeddings(nn.Module):
    """Construct the embeddings from word, segment, age
    """

    def __init__(self, config):
        super(BertEmbeddings, self).__init__()
        self.config = config
        self.activation = config.activation
        self.position_embedding_type = getattr(config, 'position_embedding_type', 'bert_embedding')
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
        self.modalities_embeddings = nn.Embedding(config.modalities_vocab_size, config.hidden_size)
        self.segment_embeddings = nn.Embedding(config.seg_vocab_size, config.hidden_size)
        
        if self.position_embedding_type == "absolute":
            self.posi_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        else:
            self.posi_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size). \
            from_pretrained(embeddings=self._init_posi_embedding(config.max_position_embeddings, config.hidden_size))

        
        if self.activation:
            self.age_embeddings = SineActivation(config.age_vocab_size, config.hidden_size)
            self.delays_embeddings = SineActivation(config.delay_vocab_size, config.hidden_size)
            
        else: 
            self.age_embeddings = nn.Embedding(config.age_vocab_size, config.hidden_size)
            self.delays_embeddings = nn.Embedding(config.delay_vocab_size, config.hidden_size)
            
        self.LayerNorm = Bert.modeling.BertLayerNorm(config.hidden_size, eps=config.layer_norm_eps)  #eps default = 1e-12
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, word_ids, modalities_ids = None, age_ids=None, delays_ids = None, seg_ids=None, posi_ids=None, age_in_inputs=False, delays_in_inputs = False):
        if seg_ids is None:
            seg_ids = torch.zeros_like(word_ids)
        if age_ids is None:
            age_ids = torch.zeros_like(word_ids)
        if posi_ids is None:
            posi_ids = torch.zeros_like(word_ids)
        if delays_ids is None:
            delays_ids = torch.zeros_like(word_ids)
        if modalities_ids is None:
            modalities_ids = torch.zeros_like(word_ids)
        
        
        word_embed = self.word_embeddings(word_ids)
        segment_embed = self.segment_embeddings(seg_ids)
        modalities_embed = self.modalities_embeddings(modalities_ids)
        age_embed = self.age_embeddings(age_ids)
        delays_embed = self.delays_embeddings(delays_ids)
        posi_embeddings = self.posi_embeddings(posi_ids)
        
        if self.config.age_in_inputs:
            embeddings = word_embed + segment_embed + delays_embed + modalities_embed + posi_embeddings
        if self.config.delays_in_inputs:
            embeddings = word_embed + segment_embed + modalities_embed + posi_embeddings
            
        else:
            embeddings = word_embed + segment_embed + posi_embeddings + modalities_embed + delays_embed + age_embed

            
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        
        return embeddings

    def _init_posi_embedding(self, max_position_embedding, hidden_size):
        def even_code(pos, idx):
            return np.sin(pos / (10000 ** (2 * idx / hidden_size)))

        def odd_code(pos, idx):
            return np.cos(pos / (10000 ** (2 * idx / hidden_size)))

        # initialize position embedding table
        lookup_table = np.zeros((max_position_embedding, hidden_size), dtype=np.float32)

        # reset table parameters with hard encoding
        # set even dimension
        for pos in range(max_position_embedding):
            for idx in np.arange(0, hidden_size, step=2):
                lookup_table[pos, idx] = even_code(pos, idx)
        # set odd dimension
        for pos in range(max_position_embedding):
            for idx in np.arange(1, hidden_size, step=2):
                lookup_table[pos, idx] = odd_code(pos, idx)

        return torch.tensor(lookup_table)


def t2v(tau, f, out_features, w, b, w0, b0, arg=None):
    if arg:
        v1 = f(torch.matmul(tau, w) + b, arg)
    else:
        #print(w.shape, t1.shape, b.shape)
        v1 = f(torch.matmul(tau, w) + b)
    v2 = torch.matmul(tau, w0) + b0
    #print(v1.shape)
    return torch.cat([v1, v2], -1)

class SineActivation(nn.Module):
    def __init__(self, in_features, out_features):
        super(SineActivation, self).__init__()
        self.out_features = out_features
        self.w0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.parameter.Parameter(torch.randn(1))
        self.w = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.b = nn.parameter.Parameter(torch.randn(out_features-1))
        self.f = torch.sin

    def forward(self, tau):
        emb = list()
        for batch in tau:
            out_list = list()
            for pos in batch: 
                out = t2v(pos.reshape(-1, 1), self.f, self.out_features, self.w, self.b, self.w0, self.b0)
                out_list.append(torch.flatten(out))
            emb.append(torch.stack(out_list, 0))
        return torch.stack(emb, 0)
    
    
class BertModel(BertPreTrainedModel):
    def __init__(self, config):
        super(BertModel, self).__init__(config)
        self.age_in_inputs = config.age_in_inputs
        self.delays_in_inputs = config.delays_in_inputs
        self.embeddings = BertEmbeddings(config=config)
        self.encoder = Bert.modeling.BertEncoder(config=config)
        self.pooler = Bert.modeling.BertPooler(config)
        self.post_init()


    def forward(self, input_ids,  modalities_ids = None, age_ids=None, delays_ids = None, seg_ids=None, posi_ids=None, attention_mask=None,
                output_all_encoded_layers=True)-> Union[Tuple[torch.Tensor], BaseModelOutputWithPoolingAndCrossAttentions]:
        if attention_mask is None:
            attention_mask = torch.ones_like(input_ids)
        if age_ids is None:
            age_ids = torch.zeros_like(input_ids)
        if seg_ids is None:
            seg_ids = torch.zeros_like(input_ids)
        if posi_ids is None:
            posi_ids = torch.zeros_like(input_ids)
        if delays_ids is None:
            delays_ids = torch.zeros_like(input_ids)
        if modalities_ids is None:
            modalities_ids = torch.zeros_like(input_ids)
     
        # We create a 3D attention mask from a 2D tensor mask.
        # Sizes are [batch_size, 1, 1, to_seq_length]
        # So we can broadcast to [batch_size, num_heads, from_seq_length, to_seq_length]
        # this attention mask is more simple than the triangular masking of causal attention
        # used in OpenAI GPT, we just need to prepare the broadcast dimension here.
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)

        # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
        # masked positions, this operation will create a tensor which is 0.0 for
        # positions we want to attend and -10000.0 for masked positions.
        # Since we are adding it to the raw scores before the softmax, this is
        # effectively the same as removing these entirely.
        extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype)  # fp16 compatibility
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

        embedding_output = self.embeddings(input_ids, modalities_ids, age_ids, delays_ids, seg_ids, posi_ids, age_in_inputs = self.age_in_inputs, delays_in_inputs = self.delays_in_inputs)
        encoded_layers = self.encoder(embedding_output,
                                      extended_attention_mask,
                                      output_all_encoded_layers=output_all_encoded_layers)
        sequence_output = encoded_layers[-1]
        pooled_output = self.pooler(sequence_output)
        if not output_all_encoded_layers:
            encoded_layers = encoded_layers[-1]
        return encoded_layers, pooled_output


class BertForMaskedLM(BertPreTrainedModel):
    def __init__(self, config):
        super(BertForMaskedLM, self).__init__(config)
        
        if config.is_decoder:
            raise ('To use BertForMaskedLM, config.is_decoder need to be set to False for bi-directional self-attention')
            
        self.bert = BertModel(config)
        self.cls = BertOnlyMLMHead(config)#, self.bert.embeddings.word_embeddings.weight)
        self.post_init()

    def forward(self, input_ids, modalities_ids = None, age_ids=None, delays_ids = None, seg_ids =None, posi_ids=None, attention_mask=None, masked_lm_labels=None)-> Union[Tuple[torch.Tensor], MaskedLMOutput]:
        outputs = self.bert(input_ids, modalities_ids, age_ids, delays_ids, seg_ids, posi_ids, attention_mask, output_all_encoded_layers=False)
        sequence_output = outputs[0]
        prediction_scores = self.cls(sequence_output)
        
        masked_lm_loss = None
        if masked_lm_labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=-1)  # -100 index = padding token
            masked_lm_loss = loss_fct(prediction_scores.view(-1, self.config.vocab_size), masked_lm_labels.view(-1))
            return masked_lm_loss, prediction_scores.view(-1, self.config.vocab_size), masked_lm_labels.view(-1)
            
        else:
            return prediction_scores
            
   #     return MaskedLMOutput(
     #       loss=masked_lm_loss,
     #       logits=prediction_scores,
    #        hidden_states=outputs.hidden_states,
    #        attentions=outputs.attentions,
    #    )
    
    

In [ ]:

""" Calculate precision for the MLM """

def cal_acc(label, pred):
    logs = nn.LogSoftmax(dim = 1)
    label=label.cpu().numpy()
    ind = np.where(label!=-1)[0]
    truepred = pred.detach().cpu().numpy()
    truepred = truepred[ind]
    truelabel = label[ind]
    truepred = logs(torch.tensor(truepred))
    outs = [np.argmax(pred_x) for pred_x in truepred.numpy()]
    precision = skm.precision_score(truelabel, outs, average='macro')
    return precision

""" Function that allows to measure elapsed time for a function """
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

""" Set seed to allow reproducibility """

def set_seed(seed_value=42):

    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    
""" Build loader for datasets """

def load_data(data, config, inputs='inputs_normal_range_and_delta_age_preprocessed_tolist', 
       mod='modalities_for_twos_and_age_preprocessed_tolist', delay= 'delays_categorized_for_twos_and_age_preprocessed_tolist',
              shuffle=False, num_workers = 3, seed=42, baseline=False):
  
    Dset = PretrainLoader(token2idx = tokenVocab,  mod2idx = modalitiesVocab, del2idx=delayVocab, max_len=train_params['max_len_seq'], dataframe=data, code=inputs, 
               mod = mod, delay=delay, delays_in_inputs=False, age_in_inputs=True, pretrain='MLM', baseline=baseline)  
    dataload = DataLoader(dataset=Dset, batch_size=config.batch_size, shuffle=shuffle, num_workers=num_workers)

    return dataload

""" Use the GPU if it's available """

if torch.cuda.is_available():
   
    device = torch.device("cuda")
    print("There are %d GPU(s) available. " %torch.cuda.device_count())
    print("We will use the GPU: {}".format(torch.cuda.get_device_name(0)))
    
else:
    print("No GPU available, using the CPU instead")
    device = torch.device("cpu")



2. Config  

In [16]:
import wandb


os.environ['https_proxy'] = ""
os.environ['http_proxy'] = ""
os.environ['no_proxy']= ""
os.environ['HTTPS_PROXY']= ""
os.environ['HTTP_PROXY']= ""
os.environ['NO_PROXY']= ""

os.environ['WANDB_NOTEBOOK_NAME'] = 'BEHRT/Early_integration/Tasks/MLM_npi_final.ipynb'
os.environ['WANDB_API_KEY'] = "4944e153a8b0f45a64d749b14d6916d253a54426"
os.environ['WANDB_NAME'] = "MLM_final_run"
os.environ['WANDB_NOTES'] = "Best hps for MLM"

wandb.login()


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find BEHRT/Early_integration/Tasks/MLM_npi_final.ipynb.
wandb: Currently logged in as: ndeyemaguettemb (ehr). Use `wandb login --relogin` to force relogin


True

3. Import files

In [18]:
file_config = {'data': 'whole_data_preprocessed_v2_for_pretrain_exploded.pkl', 
    'model_path': 'BEHRT/Early_integration/Tasks/Output/Second_run', # where to save model
    'model_name': 'MLM_embeddings_final', # model name
    'file_name': 'MLM_embeddings_final',  # log path
}
create_folder(file_config['model_path'])  ## on crée un folder ou stocker le modele 

global_params = {
    'max_seq_len': 512,
    'max_age': 110,
    'max_delay': 30, ## = 30years
    'age_month': 12,
    'delay_month': 0.25,
    'age_symbol': None,
    'min_visit': 1,
}

train_params = {
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'device': device,
}


#### Import dataset

In [19]:
data = hf._load_pkl(file_config['data'])[0]

,Unnamed: 0,age_100_therap_removed_into_chunks_tolist,delays_100_therap_removed_into_chunks_tolist,modalities_100_therap_removed_into_chunks_tolist,inputs_quantiles_preprocessed_100_therap_removed_tolist,inputs_normal_range_preprocessed_100_therap_removed_tolist,inputs_curve_intersections_preprocessed_100_therap_removed_tolist,report,Num_dossier
0,0,"['61', '61', '61', '61', '61', '61', '61', '61...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['DEB', 'biologique', 'biologique', 'biologiqu...","['CLS', '1.0', '1.0', '1.0', '1.0', '1.0', '1....","['CLS', '1.0', '1.0', '2.0', '2.0', '1.0', '1....","['CLS', '1.0', '1.0', '1.0', '1.0', '1.0', '1....","[""\nCorps du document\n\n\n\n\n\n\n\n\n\n16/07...",00002f6df9cfcb97985c88f5787ba874
1,1,"['42', '42', '42', '42', '42', '42', '42', '42...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['DEB', 'biologique', 'biologique', 'biologiqu...","['CLS', '0.0', '0.0', '0.0', '0.0', '0.0', '0....","['CLS', '0.0', '0.0', '0.0', '0.0', '0.0', '0....","['CLS', '0.0', '0.0', '0.0', '0.0', '0.0', '0....","[""\nCorps du document\n\n\n\n\n\n\n\n\n\n\n23/...",00013328b341eedd7d4d788140b765a2
2,2,"['45', '45', '45', '45', '45', '45', '45', '45...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['DEB', 'biologique', 'biologique', 'biologiqu...","['CLS', '0.0', '0.0', '0.0', '0.0', '0.0', '0....","['CLS', '0.0', '0.0', '0.0', '0.0', '0.0', '0....","['CLS', '0.0', '0.0', '0.0', '0.0', '0.0', '0....","[""\nCorps du document\n\nours en cours. Il s'...",000ad3d5781425daf543e1cf0e099ec6
3,3,"['46', '46', '46', '46', '46', '46', '46', '46...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['DEB', 'biologique', 'biologique', 'biologiqu...","['CLS', '0.0', '0.0', '0.0', '0.0', '0.0', 'co...","['CLS', '0.0', '0.0', '0.0', '0.0', '0.0', 'co...","['CLS', '0.0', '0.0', '0.0', '0.0', '0.0', 'co...","[""\nCorps du document\n\nPatiente née le 20/07...",000c21bab84929a0029ade6db24d5b75
4,4,"['49', '49', '49', '49', '49', '49', '49', '49...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['DEB', 'biologique', 'biologique', 'biologiqu...","['CLS', '0.0', '0.0', '0.0', '0.0', '0.0', 'ra...","['CLS', '0.0', '0.0', '0.0', '0.0', '0.0', 'ra...","['CLS', '0.0', '0.0', '0.0', '0.0', '0.0', 'ra...",['\nCorps du document\n\nMammographie de ville...,00143b158e6cad759fddbd194f7ed284


In [ ]:
train, valid = import_datasets(file_config, inputs = 'inputs_normal_range_and_delta_age_preprocessed_tolist', min_visit= global_params['min_visit'] )
print(train.shape, valid.shape)             

In [20]:
train.index = range(len(train))
valid.index = range(len(valid))

We lost 0 patient


In [ ]:
hf._dump_pkl(train.index, 'BEHRT/Early_integration/Files/MLM_train_idxs')
hf._dump_pkl(valid.index, 'BEHRT/Early_integration/Files/MLM_valid_idxs')

a. Compute vocabularies

In [ ]:
inputs = "inputs_normal_range_and_delta_age_preprocessed_tolist"
modalities = "modalities_for_twos_and_age_preprocessed_tolist"

data = pd.concat([train, valid])
categorized = True

tokenVocab, _ = input_vocab(inputs = data[inputs], symbol=global_params['age_symbol'])
modalitiesVocab, _ = mod_vocab(data[modalities], symbol=global_params['age_symbol'])
delayVocab = {'PAD':0, 'UNK':1, 'W0':2, 'W1':3, 'W2':4, 'W3':5, 'M1':6, 'M2':7, 'M3':8, 'M4':9, 'M5':10, 'M6':11, 'M7':12, 'M8':13, 'M9':14, 'M10':15, 'M11':16, 'M12':17, 'LT':18}


ValueError: list.remove(x): x not in list

b. Remove patients with less visits than minimum visit (1)

In [ ]:
# Remove patients with visits less than min visit

previous_shape = data.shape[0]

data['length'] = data[inputs].apply(lambda x: len([i for i in range(len(x)) if x[i] == 'SEP']))
data = data[data['length'] >= global_params['min_visit']]
data = data.reset_index(drop=True)

print("We lost {} patient(s)".format(previous_shape - data.shape[0]))

### Model

In [ ]:
from Utils import optimiser

def define_model(config):
    conf = BertConfig(config)
    model = BertForMaskedLM(conf)
    model = model.to(train_params['device'])
    optim = optimiser.adam(params=list(model.named_parameters()), config=config)
    
    return model,optim

In [19]:

## Config for MLM with sweep (wandb)

from transformers.configuration_utils import PretrainedConfig
        
class BertConfig(PretrainedConfig):
    def __init__(self, config):
        super(BertConfig, self).__init__(
            vocab_size=config.get('vocab_size'),
            hidden_size=config.get('hidden_size'),
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings = config.get('max_position_embeddings'),
            position_embedding_type = config.get('position_embedding_type'),
            initializer_range=config.get('initializer_range'),
            layer_norm_eps = config.get('layer_norm_eps'),
            output_hidden_states=config.get('output_hidden_states'),
            
            
        )
        self.delay_vocab_size = config.get('delay_vocab_size')
        self.output_hidden_states=config.get('output_hidden_states')
        self.modalities_vocab_size = config.get('modalities_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.delay_vocab_size= config.get('delay_vocab_size')
        self.activation = config.get('activation')
        self.output_attentions = config.get('output_attentions')
        self.chunk_size_feed_forward = config.get('chunk_size_feed_forward')
        self.is_decoder = config.get('is_decoder')
        self.layer_norm_eps = config.get('layer_norm_eps')
        self.add_cross_attention = config.get('add_cross_attention')
        self.age_in_inputs = config.get('age_in_inputs')
        self.delays_in_inputs = config.get('delays_in_inputs')
        self.batch_size = config.get('batch_size')
       
        
class TrainConfig(object):
    def __init__(self, config):
        self.batch_size = config.get('batch_size')
        self.use_cuda = config.get('use_cuda')
        self.max_len_seq = config.get('max_len_seq')
        self.train_loader_workers = config.get('train_loader_workers')
        self.test_loader_workers = config.get('test_loader_workers')
        self.device = config.get('device')
        self.output_dir = config.get('output_dir')
        self.output_name = config.get('output_name')
        self.best_name = config.get('best_name')
        

In [18]:
## Config for MLM with sweep (wandb)

config = {'hidden_size':  144,       # word embedding and seg embedding hidden size
                'intermediate_size': 133,                # the size of the "intermediate" layer in the transformer encoder
                'hidden_dropout_prob': 0.2,                    # dropout rate
                'num_hidden_layers': 5,                   # number of multi-head attention layers required
                'num_attention_heads': 12,                  # number of attention heads
                'attention_probs_dropout_prob':0.2,
                'hidden_act': 'gelu',                                                 # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
                'vocab_size': len(tokenVocab),                                                  # number of disease + symbols for word embedding
                'seg_vocab_size': 2,                                                             # number of vocab for seg embedding
                'modalities_vocab_size': len(modalitiesVocab), 
                'age_vocab_size': 1,                                                 # number of vocab for age embedding
                'delay_vocab_size': len(delayVocab),
                'epochs': 120, 
                'max_position_embeddings': train_params['max_len_seq'],                          # maximum number of tokens
                'initializer_range':0.02,                                                     # parameter weight initializer range
                'lr' : 0.001,                     # a flat distribution between 0 and 0.1
                'warmup_proportion': 0.01, 
                'weight_decay' : 0.01, 
                'batch_size': 64,   # integers between 8 and 32 # with evenly-distributed logarithm
                'gradient_accumulation_steps': 4,    
                'layer_norm_eps': 0.00001,
                'position_embedding_type': 'bert_embedding', 
                'age_in_inputs': True,
                'delays_in_inputs':False,
                'activation': False,
                'counts': 5}
                #'early_terminate': {'type': 'hyperband', 's': 2, 'eta': 3, 'max_iter': 27},



NameError: name 'tokenVocab' is not defined

In [ ]:
## Initiate model with new parameters (useful for new runs)

def initiate_model(config):
    conf = BertConfig(config)
    model = BertForMaskedLM(conf)
    model = model.to(train_params['device'])
    optim = adam(params=list(model.named_parameters()), config=config)
    
    return model, optim

# save
def save(model, optimizer, count):
    # save
    torch.save({
        f'model_state_dict_{count}': model.state_dict(),
        f'optimizer_state_dict_{count}': optimizer.state_dict()
    }, output_model)


In [ ]:
from pytorch_pretrained_bert import WEIGHTS_NAME, CONFIG_NAME

def empty_cuda():
    """ Empty the cache to enable the use of the gpu """
    torch.cuda.empty_cache()
    print(torch.cuda.memory_summary(device=None, abbreviated=False))

    

def training_per_epoch(e, model, optim, config, loader, train_params):
        
        output_dir = "./pretrained_models/"
        create_folder(output_dir)  ## on crée un folder ou stocker le modele 
        
        tr_loss, temp_loss = [], []
        precision, temp_precision = [], []
        nb_tr_examples, nb_tr_steps = 0, 0
        cnt= 0
        start = time.time()

        model.train()
        
        for step, batch in enumerate(loader):

            cnt +=1
                
            batch = tuple(t.to(train_params['device']) for t in batch)
            
            if config.age_in_inputs:
                input_ids, mod_ids, del_ids, segment_ids, posi_ids, attMask, masked_label = batch
                age_ids = None
        
             #   age_ids = age_ids.to(train_params['device'])
                
            elif config.delays_in_inputs:
                input_ids, mod_ids, segment_ids, posi_ids, attMask, masked_label = batch
                age_ids = None
              #  age_ids = age_ids.to(train_params['device'])
                
                del_ids = None
               # del_ids = del_ids.to(train_params['device'])
                
            else:
                input_ids, mod_ids, age_ids, del_ids, segment_ids, posi_ids, attMask, masked_label = batch


            loss, pred, label = model(input_ids=input_ids, modalities_ids=mod_ids, delays_ids=del_ids, seg_ids=segment_ids, posi_ids=posi_ids, attention_mask=attMask, masked_lm_labels=masked_label)

            if config.gradient_accumulation_steps >1:
                loss = loss/config.gradient_accumulation_steps

            loss.backward()
            temp_loss.append(loss.item())
            tr_loss.append(loss.item())

            nb_tr_examples += input_ids.size(0)
            nb_tr_steps += 1
            
            if step % 100 == 0:
                with open("mydocument.txt", mode = "a") as f:
                    print("epoch: {}\t| cnt: {}\t|Loss: {}\t| precision: {:.4f}\t| time: {:.2f}".format(e, cnt, np.mean(temp_loss), cal_acc(label, pred), time.time()-start))
                    temp_loss = []
                    start = time.time()

            if (step + 1) % config.gradient_accumulation_steps == 0:
                optim.step()
                optim.zero_grad()

        # print("** ** * Saving fine - tuned model ** ** * ")
        model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
        create_folder(file_config['model_path'])
        
        output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
        torch.save(model_to_save.state_dict(), output_model_file)
        
        cost = time.time() - start

        return np.mean(tr_loss), cost, cal_acc(label, pred), model_to_save
    
def evaluate_per_epoch(model, loader, config, train_params):
    
    temp_loss, val_loss = [], []
    nb_tr_examples, nb_tr_steps = 0, 0
  
    start = time.time()
 
    model.eval()
    
    for step, batch in enumerate(loader):
        
        
        batch = tuple(t.to(train_params['device']) for t in batch)

        if config.age_in_inputs:
            input_ids, mod_ids, del_ids, segment_ids, posi_ids, attMask, masked_label = batch
            age_ids = None

        elif config.delays_in_inputs:
            input_ids, mod_ids, segment_ids, posi_ids, attMask, masked_label = batch
            age_ids = None
            del_ids = None

        else:
            input_ids, mod_ids, age_ids, del_ids, segment_ids, posi_ids, attMask, masked_label = batch

        
        with torch.no_grad():
            loss, pred, label = model(input_ids = input_ids, modalities_ids = mod_ids, age_ids=age_ids,delays_ids=del_ids, seg_ids=segment_ids, posi_ids=posi_ids, attention_mask=attMask, masked_lm_labels=masked_label)
            loss = loss.cpu()
            val_loss.append(loss)
            
        temp_loss.append(loss)
        
        if step % 50 == 0:
            print("Validation:")
            print("|Val Loss: {}\t| Val precision: {:.4f}\t| time: {:.2f}".format(np.mean(temp_loss), cal_acc(label, pred), time.time()-start))
            temp_loss = []
            start = time.time() 
            
    return cal_acc(label, pred), val_loss, np.mean(val_loss)


In [ ]:

def training(config):
    counter_dict, valid_counter_dict, embedding_dict, models_dict = dict(), dict(), dict(), dict()
    temp_loss = 1
    with wandb.init(config=config, project="MLM_final_run") as run:
        config = wandb.config
        
        for count in [0, 1, 2, 3, 'baseline', 'baseline_2']:

             # Copy your config 
            run.name = f"MLM_final_run_{count}"
            
            if isinstance(count, int):
                seed = count
            else:
                seed = random.randrange(0, 100)

            results_dict = {'epochs': range(config.epochs), 'loss': [] , 'precision':[]}
            valid_results_dict = {'epochs': range(config.epochs), 'loss': [] , 'precision':[]}


            if count in ['baseline', 'baseline_2']:
                train_loader = load_data(train, config, seed=seed, baseline=True)
                valid_loader = load_data(valid, config, seed=seed, baseline= True)
            else: 
                train_loader = load_data(train, config, seed=seed)
                valid_loader = load_data(valid, config, seed=seed)

            temp_prec, val_temp_prec = [], []

            model, optim = define_model(config)

            for epoch in range(config.epochs):
                loss, time_cost, precision, model_to_save = training_per_epoch(e=epoch, loader=train_loader, model=model, optim=optim, config=config, train_params= train_params)
                val_precision, val_loss_tot, val_loss = evaluate_per_epoch(model= model, loader = valid_loader, config= config, train_params=train_params)

                results_dict['loss'].append(loss)
                results_dict['precision'].append(precision)

                valid_results_dict['loss'].append(val_loss)
                valid_results_dict['precision'].append(val_precision)

                if epoch >= config.epochs - 20:
                    temp_prec.append(precision)
                    val_temp_prec.append(val_precision)


                wandb.log({"epoch" : epoch,
                           "prec": precision,
                            "loss" : loss,
                           "val_prec" : val_precision,
                           "val_loss": val_loss})

            torch.save(model_to_save, f"MLM_model_{count}_{run.id}")

            wandb.log({"20_last_prec": np.mean(temp_prec),
                "20_val_last_prec": np.mean(val_temp_prec)})
            
            if epoch%5 == 0:
       #     temp_f1, temp_auc, temp_aps, temp_recall, temp_precision, temp_loss, temp_time, tem_hidden_states, temp_masks = evaluating(model, config, testload, global_params, mlb)
                if temp_loss > loss: 
          #      wandb.alert(title= "Temp auc is high", text=f"Temp auc is : {temp_auc}")
                    torch.save(model_to_save, f'BEHRT/Early_integration/Outs_bis/pretrained_for_{count}_{temp_loss}')
                    temp_loss = loss  
                    wandb.log({'temp_loss': temp_loss, 'temp_epoch':epoch})
                    


        #    if (np.mean(temp_prec) >= 0.5) | (np.mean(val_temp_prec) >=0.5):
            hf._dump_pkl(results_dict, f"BEHRT/Early_integration/Outs/results_dict_{count}_{run.id}")
            hf._dump_pkl(valid_results_dict, f"BEHRT/Early_integration/Outs/valid_results_dict_{count}_{run.id}")


            wandb.alert(
                title=f"Terminate run", 
                text=f"Run finished with val : {val_precision}",
                )

            counter_dict[count] = results_dict
            valid_counter_dict[count] = valid_results_dict
            embedding_dict[count] = model_to_save.state_dict()['bert.embeddings.word_embeddings.weight'].cpu()  
            models_dict[count] = model_to_save

        wandb.finish()

    return counter_dict, valid_counter_dict, embedding_dict, models_dict
          


##### Training 

In [ ]:
if __name__ == "__main__":

    
    print("Running Masked Language Model")  
    print("*"*15+" Run n° "+str(1)+" "+"*"*15)
    print()
    
    start = time.time()   
    
   # training(config = config)
    counter_dict, valid_counter_dict, embedding_dict, models_dict = training(config = config)
    
    print()
    print('Elapsed time for sweep hyperparameters tuning = {}"'.format(time.time() - start))
    

In [75]:
hf._dump_pkl(counter_dict, 'BEHRT/Early_integration/Tasks/Output/counter_dict_completed_3')
hf._dump_pkl(valid_counter_dict, 'BEHRT/Early_integration/Tasks/Output/valid_counter_dict_completed_3')
hf._dump_pkl(embedding_dict, 'BEHRT/Early_integration/Tasks/Output/embedding_dict_completed_3')
hf._dump_pkl(models_dict, 'Model_pretraineds_3')

The file has been saved in pkl format in /home/jovyan/work/Workdir/Secondment_1/Secondment 1/tasks/Early integration/BEHRT Final version
The file has been saved in pkl format in /home/jovyan/work/Workdir/Secondment_1/Secondment 1/tasks/Early integration/BEHRT Final version
